# 决策树简介

# 熵
+ 表示随机变量的不确定性  
$H(p)=-\sum_{i=1}^{n} p_{i} \log p_{i}$
+ 熵越大，随机变量的不确定性越大！！

# 熵的计算
$H(p)=-\sum_{i=1}^{n} p_{i} \log p_{i}$  
$H(D)=-\sum_{k=1}^{k} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}$

# 条件熵
定义如下：
$H(Y \mid X)=\sum_{i=1}^{n} p_{i} H\left(Y \mid X=x_{i}\right)$  
其中 $p_{i}=P\left(X=x_{i}\right)$    

---
$H(D \mid A)=\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{D \mid} H\left(D_{i}\right)$  
$=-\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{D \mid} \sum_{k=1}^{K} \frac{\left|D_{ik}\right|}{D_{i} \mid} \log _{2} \frac{\left|D_{ik}\right|}{\left|D_{i}\right|}$

# 信息增益  
+ 定义  
信息增益（也叫互信息），定义如下：  
$g(D,A) = H(D)-H(D|A)$  其中D是训练数据集，A是某个特征
+ 根据信息增益准则的特征选择方法是：
    1. 对训练数据集（或子集）D， 计算其中每个特征的洗洗增益
    2. 比较他们的大小，选择信息增益最大的特征
+ 信息增益法
    1. 计算数据集D的经验熵$H(D)$:   
    $H(D)=-\sum_{k=1}^{k} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}$
    2. 计算特征A对数据集的经验条件熵$H(D|A)$  
    $H(D \mid A)=\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} H\left(D_{i}\right)=-\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} \sum_{k=1}^{k} \frac{\left|D_{ik}\right|}{\left|D_{i}\right|} \log _{2} \frac{\left|D_{ik}\right|}{\left|D_{i}\right|}$
    3. 计算信息增益   
    $g(D,A) = H(D)-H(D|A)$


+ 例子  
![](./img/4_1.png)

+ ID3算法： 在决策树递归构建过程中，使用信息增益的方法进行特征选择
+ 决策树生成过程：  
 1. 从根节点开始计算所有特征的信息增益，选择信息增益最大的特征作为结点特征
 2. 再对子节点递归以上的方法，构建决策树
 3. 所有特征信息增益很小或没有特征可以选择时递归结束得到一颗决策树

# ID3代码实现
![](./img/4_2.png)

In [1]:
def loaddata():
    feature_name = ['age','job','house','credit']
    dataSet = [[0, 0,0,0, 'no'],
               [0, 0,0,1,'no'],
               [0, 1,0,1, 'yes'],
               [0, 1,1,0, 'yes'],
               [0, 0,0,0, 'no'],
               [1, 0,0,0, 'no'],
               [1, 0,0,1, 'no'],
               [1, 1,1,1, 'yes'],
               [1, 0,1,2, 'yes'],
               [1, 0,1,2, 'yes'],
               [2, 0,1,2, 'yes'],
               [2, 0,1,1, 'yes'],
               [2, 1,0,1, 'yes'],
               [2, 1,0,2, 'yes'],
               [2, 0,0,0,'no']]
    
    return dataSet, feature_name
datas, features = loaddata()
datas[:3]

[[0, 0, 0, 0, 'no'], [0, 0, 0, 1, 'no'], [0, 1, 0, 1, 'yes']]

In [22]:
from scipy import math
def entropy(datas):
    entropy_ = 0
    label_counts = {}
    goaldata = [d[-1] for d in datas]
    length = len(datas)
    for d in goaldata:
        label_counts[d] = label_counts.get(d, 0) + 1
    
    for k, v in label_counts.items():
        p = v / length
        e_ = - p * math.log2(p)
        entropy_ += e_
    return entropy_

def feature_entropy(datas, axis):
    all_length = len(datas)
    feature_entropy = 0
    discounts = set([d[axis] for d in datas])
    for dis in discounts:
        data = [d for d in datas if d[axis] == dis]
        data_length = len(data)
        e = (data_length / all_length) * entropy(data)
        feature_entropy += e
    return feature_entropy

def choose_best_feature(datas, features):
    all_e = entropy(datas)
    info_gain = []
    for idx, feature in enumerate(features):
        feature_e = feature_entropy(datas, idx)
        diff = all_e - feature_e
        info_gain.append(diff)
    
    max_ = max(info_gain)
    idx = info_gain.index(max_)
    feature = features[idx]
    
    return idx, feature
    

#划分数据集
def split_dataset(datas, features, axis, value):
    ret_dataset = [d[0:axis] + d[axis+1:] for d in datas if d[axis] == value]
    return ret_dataset

def class_vote(class_list):
    class_set = set(class_list)
    class_count = {c: class_list.count(c) for c in class_set}
    class_count = sorted(class_count.items(), key=lambda d: d[1], reverse=True)
    return class_count[0][0]

def train_tree(datas, features):
    classlist = [d[-1] for d in datas]
    if len(set(classlist)) == 1: #就剩下一个类别
        return classlist[0]
    if len(datas[0]) == 1: #就剩下一个特征
        return class_vote(datas)
    
    bestidx, bestfeature = choose_best_feature(datas, features)
    print(bestfeature, features)
    best_datas = [d[bestidx] for d in datas]
    my_tree = {bestfeature:{}}
    for value in set(best_datas):
        datas = split_dataset(datas, features, bestidx, value)
        features.remove(bestfeature)
        my_tree[bestfeature][value] = train_tree(datas, features)
        
    return my_tree
        

In [23]:
myDat,feature_name = loaddata()
myTree = train_tree(myDat,feature_name)
print(myTree)

house ['age', 'job', 'house', 'credit']
job ['age', 'job', 'credit']


ValueError: list.remove(x): x not in list